# LOB Dataset for Projects

This jupyter notebook is used to download the FI-2010 [1] dataset for train and test a AI classifier on LOB data. 
The code is obtained from [2].

### Data:
The FI-2010 is publicly avilable and interested readers can check out their paper [1]. The dataset can be downloaded from: https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649 

Otherwise, the notebook will download the data automatically or it can be obtained from: 

https://drive.google.com/drive/folders/1Xen3aRid9ZZhFqJRgEMyETNazk02cNmv?usp=sharing

### References:
[1] Ntakaris A, Magris M, Kanniainen J, Gabbouj M, Iosifidis A. Benchmark dataset for mid‐price forecasting of limit order book data with machine learning methods. Journal of Forecasting. 2018 Dec;37(8):852-66. https://arxiv.org/abs/1705.03233

[2] Zhang Z, Zohren S, Roberts S. DeepLOB: Deep convolutional neural networks for limit order books. IEEE Transactions on Signal Processing. 2019 Mar 25;67(11):3001-12. https://arxiv.org/abs/1808.03668

### This notebook runs on Pytorch 1.9.0.

In [1]:
# download the data
import os 
if not os.path.isfile('data.zip'):
    !wget https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
    !unzip -n data.zip
    print('data downloaded.')
else:
    print('data already existed.')

data already existed.


In [2]:
!ls -la

total 973112
drwxr-xr-x  6 umberto umberto      4096 Jan 26 20:07 .
drwxr----- 96 umberto umberto      4096 Jan 25 13:40 ..
drwxr-xr-x 33 umberto umberto      4096 Jan 26 19:38 ai4t-DeepLobster
-rw-r--r--  1 umberto umberto     40782 Jan 26 20:07 dataset_download.ipynb
-rw-r--r--  1 umberto umberto  56278154 Dec 29 11:46 data.zip
drwxr-xr-x  7 umberto umberto      4096 Jan 24 13:49 .git
drwxr-xr-x  2 umberto umberto      4096 Dec 29 11:47 .ipynb_checkpoints
-rw-r--r--  1 umberto umberto 132259850 Jul 14  2021 Test_Dst_NoAuction_DecPre_CF_7.txt
-rw-r--r--  1 umberto umberto 124378346 Jul 14  2021 Test_Dst_NoAuction_DecPre_CF_8.txt
-rw-r--r--  1 umberto umberto  76138106 Jul 14  2021 Test_Dst_NoAuction_DecPre_CF_9.txt
-rw-r--r--  1 umberto umberto 607324298 Jul 14  2021 Train_Dst_NoAuction_DecPre_CF_7.txt
drwxr-xr-x 53 umberto umberto      4096 Jan 26 19:38 wandb


In [1]:
# load packages
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report

# Data preparation

We used no auction dataset that is normalised by decimal precision approach in their work. The first seven days are training data and the last three days are testing data. A validation set (20%) from the training set is used to monitor the overfitting behaviours.  

The first 40 columns of the FI-2010 dataset are 10 levels ask and bid information for a limit order book and we only use these 40 features in our network. The last 5 columns of the FI-2010 dataset are the labels with different prediction horizons. 

In [2]:
# please change the data_path to your local path
# data_path = '/nfs/home/zihaoz/limit_order_book/data'
dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt') # 80 training - 20 validation
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 139587)


In [3]:
# all the data refer to 7 days, and the first 5 days are in the training set and validation 
# and the last 2 days are inside the test set 
x_training_data = dec_train.T[:, :40]
x_validation_data = dec_val.T[:, :40]
x_test_data = dec_test.T[:, :40]

In [4]:
print(x_training_data.shape, x_validation_data.shape, x_test_data.shape)

(203800, 40) (50950, 40) (139587, 40)


In [5]:
x_training_data[0] # 40 --> 10 levels and (ask-price, ask-volume, bid-price, bid-volume)

array([0.2615 , 0.00353, 0.2606 , 0.00326, 0.2618 , 0.002  , 0.2604 ,
       0.00682, 0.2619 , 0.00164, 0.2602 , 0.00786, 0.262  , 0.00532,
       0.26   , 0.00893, 0.2621 , 0.00151, 0.2599 , 0.00159, 0.2623 ,
       0.00837, 0.2595 , 0.001  , 0.2625 , 0.0015 , 0.2593 , 0.00143,
       0.2626 , 0.00787, 0.2591 , 0.00134, 0.2629 , 0.00146, 0.2588 ,
       0.00123, 0.2633 , 0.00311, 0.2579 , 0.00128])

In [6]:
x_training_data[1]  # second time instant

array([0.2615 , 0.00211, 0.2606 , 0.00326, 0.2619 , 0.00164, 0.2604 ,
       0.00682, 0.262  , 0.00138, 0.2602 , 0.00786, 0.2621 , 0.00545,
       0.2601 , 0.00393, 0.2625 , 0.0015 , 0.26   , 0.005  , 0.2626 ,
       0.00787, 0.2599 , 0.00159, 0.2629 , 0.00146, 0.2595 , 0.001  ,
       0.2633 , 0.00311, 0.2593 , 0.00143, 0.2637 , 0.00165, 0.2591 ,
       0.00134, 0.2646 , 0.00138, 0.2588 , 0.00123])

In [7]:
TARGET_CLS_IDX = -1
assert -5 <= TARGET_CLS_IDX <= -1
y_training_data = dec_train.T[:,TARGET_CLS_IDX]
y_validation_data = dec_train.T[:,TARGET_CLS_IDX]
y_test_data = dec_train.T[:,TARGET_CLS_IDX]
print(y_training_data.shape)

(203800,)


In [8]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY




#### Dataset info:

The 'x' is an 2d-array that contains, for each row a snapshot of the orderbook in the following structure:
'best-ask price', 'best-ask volume', 'best-bid price', 'best-bid volume', '2-lev ask price', '2-levl ask volume', '2-lev bid price', '2-lev bid volume', ....


In [9]:
import torch

def get_window_indices(
    n_elements: int, window_length: int
):
    """
    Constructs the list of window indices used to slice input and target tensors in
    the Dataset class.
    """
    return [
        range(i, i + window_length)
        for i in range(n_elements - window_length)
    ]#[::window_length]


In [10]:
from torch.utils.data import Dataset, DataLoader
from einops import rearrange

class TheirDataset(Dataset):
    def __init__(self, data, k, num_classes, T):
        self.k = k
        self.num_classes = num_classes
        self.T = T
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)
    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]
    def __len__(self):
        """Denotes the total number of samples"""
        return self.length
    
    

class LobDataset(Dataset):
    def __init__(self, inputs, labels, window_len):
        self.inputs = (torch.from_numpy(inputs).float()).transpose(0,1)
        self.labels = torch.from_numpy(labels).long() - 1
        self.window_length = window_len
        self._initial_window_indices = torch.tensor(
            get_window_indices(
                n_elements=inputs.shape[0],
                window_length=self.window_length,
            ),
            dtype=torch.long,
        )[:-1]
        
        self.window_indices = self._initial_window_indices.clone()


    def reset(self):
        self.window_indices = self._initial_window_indices + torch.randint_like(
            self._initial_window_indices, low=0, high=self.window_length - 1
        )

    def __getitem__(self, i):
        indices = self.window_indices[i]
        return {'inputs': self.inputs[:,indices], 
                'labels': self.labels[indices[-1]]
        }
    
    def __len__(self):
        return len(self.window_indices)
    

their_dataset = TheirDataset(data=dec_train, k=4, num_classes=3, T=100)
# train_dataset = LobDataset(data=dec_train, k=4, num_classes=3, T=100)
# val_dataset = LobDataset(data=dec_val, k=4, num_classes=3, T=100)
# test_dataset = LobDataset(data=dec_test, k=4, num_classes=3, T=100)

train_dataset = dataset = LobDataset(x_training_data, y_training_data, window_len=100)
val_dataset = LobDataset(x_validation_data, y_validation_data, window_len=100)
test_dataset = LobDataset(x_test_data, y_test_data, window_len=100)
# print(dataset[0]['inputs'].shape, dataset[0]['labels'].shape)
print(len(dataset))

203699


In [11]:
elem = dataset[0]
seq, labels = elem.values()

In [12]:
their_elem = their_dataset[0]
their_seq, their_labels = their_elem
seq.shape, labels.shape, their_seq.shape, their_labels.shape

(torch.Size([40, 100]),
 torch.Size([]),
 torch.Size([1, 100, 40]),
 torch.Size([]))

In [14]:
from torch import nn
from torch.nn import functional as F
from einops.layers.torch import Rearrange


class ResBlock(nn.Module):
    def __init__(self, dim, kernel_size, activation = nn.LeakyReLU(0.01), dropout=0.):
        super().__init__()
        dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        self.arch = nn.Sequential(
            nn.Conv1d(dim, dim // 2, kernel_size=1, padding='same'),
            nn.BatchNorm1d(dim // 2),
            dropout,
            activation,
            nn.Conv1d(dim // 2, dim // 2, kernel_size=kernel_size, padding='same'),
            nn.BatchNorm1d(dim // 2),
            dropout,
            activation,
            nn.Conv1d(dim // 2, dim, kernel_size=1, padding='same'),
            nn.BatchNorm1d(dim),
            dropout,
            activation,
        )
    def forward(self, x):
        return x + self.arch(x)

class LobCNN(nn.Module):
    def __init__(self):
        super().__init__()
        activation = nn.LeakyReLU(0.01)
        self.conv1 = nn.Sequential(
            # nn.Conv2d(1, 16, kernel_size=(5,40),stride=(1,40),padding=(2,0)),
            # activation,
            # nn.BatchNorm2d(16),
            nn.Conv1d(40, 20, kernel_size=1, groups = 10, padding='same'),
            nn.Conv1d(20, 10, kernel_size=1, groups = 10, padding='same'),
        )
            
        self.arch = nn.Sequential(
            # [ Batch, C, Seq ]
            nn.Conv1d(10,32, kernel_size=(2,),stride=2),
            nn.BatchNorm1d(32),
            activation,
            # [ Batch, C, Seq // 2 ]
            nn.Conv1d(32,64, kernel_size=(2,),stride=2),
            nn.BatchNorm1d(64),
            activation,
            ResBlock(64, (2,),dropout=0.2),
            # [ Batch, C, Seq // 2 ]
            nn.Conv1d(64,64, kernel_size=(2,),stride=2),
            # [ Batch, 32, Seq // 2 ]
            nn.Conv1d(64,128, kernel_size=(2,),padding='same'),
            nn.BatchNorm1d(128),
            activation,
            ResBlock(128, (2,),dropout=0.2),
            nn.Conv1d(128,256, kernel_size=(2,),stride=(2,)),
            nn.BatchNorm1d(256),
            activation,
            nn.Conv1d(256,256, kernel_size=(2,),stride=(2,)),
            # # [ Batch, 32, Seq // 4 ]
            # nn.Conv1d(64,64, kernel_size=(3,)),
            # nn.BatchNorm1d(64),
            # activation,
            # # [ Batch, 32, Seq // 8 ]
            # nn.MaxPool1d(2),
            # [ Batch, 32 * Seq // 16 ] = [Batch, 2 * Seq]
            Rearrange('batch chan seq -> batch (chan seq)'),
            nn.Linear(768, 768),
            activation,
            nn.Linear(768,3)
        )
    def forward(self, x):
        conved = self.conv1(x)
        conved = conved.squeeze()
        return self.arch(conved)
        
class DynamicWeightCrossEntropy(nn.Module):
    def __init__(self, n_classes, decay=0.9, minimum_weight=0.05, label_smoothing=0.0):
        super().__init__()
        self.n_classes = n_classes
        weight = torch.ones(n_classes, dtype=torch.float, requires_grad=False)
        self.register_buffer("weight", weight)
        self.decay = decay
        self.minimum_weight = minimum_weight
        self.label_smoothing = label_smoothing

    def forward(self, logits, targets):
        value_counts = torch.ones_like(self.weight)
        # is there a better way to do this? it doesn't really hurt performance though
        for i in range(self.n_classes):
            value_counts[i] += (targets == i).sum()
        # obtain the inverse value counts
        new_weight = (value_counts.sum()) / value_counts
        # normalize so it sums to 1
        new_weight = new_weight / new_weight.sum()
        # cap the minimum weight to prevent weights that are too small (leads to nan/infinity)
        new_weight = torch.maximum(new_weight, torch.tensor(self.minimum_weight))
        # re-normalize
        new_weight = new_weight / new_weight.sum()
        # update weights with smoothing
        new_weight = (self.decay) * self.weight + (1 - self.decay) * new_weight
        # trick to only update weight estimation during training
        if logits.requires_grad:
            self.weight = new_weight
        return F.cross_entropy(
            logits,
            targets,
            weight=new_weight,
            reduction="mean",
            label_smoothing=self.label_smoothing,
        )

        

In [15]:
import pytorch_lightning as pl
from torchmetrics import functional as M

class LobLightningModule(pl.LightningModule):
    def __init__(self, train_dataset, val_dataset, test_dataset, lr=1e-3, weighted_loss=False):
        super().__init__()
        self.save_hyperparameters()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        
        self.model = LobCNN()
        self.loss = (
            nn.CrossEntropyLoss()
            if not weighted_loss
            else
            DynamicWeightCrossEntropy(n_classes=3, decay=0.99,minimum_weight=0.1)
        )
    
    def forward(self, inputs, **kwargs):
        out = {'model_out': self.model.forward(inputs) }
        if 'labels' in kwargs:
             out['labels'] = kwargs['labels']
        return out
        
    
    def step(self, inputs, labels, mode):
        model_out = self.model(inputs) # :: [ Batch Classes ]
        predictions = model_out
        loss = self.loss(model_out, labels.view(-1))
        self.log(f'{mode}/loss', loss)
        return {'model_out': model_out, 'loss': loss, 'labels':labels}
    
    def training_step(self, batch, batch_idx):
        return self.step(**batch, mode='train')
    
    def validation_step(self, batch, batch_idx):
        return self.step(**batch, mode='val')
    
    def test_step(self, batch, batch_idx):
        return self.step(**batch, mode='test')
    
    def epoch_end(self, step_outputs, mode):
        model_outs = torch.cat([ x['model_out'] for x in step_outputs],dim=0)
        labels = torch.cat([ x['labels'] for x in step_outputs], dim=0)
        
        accuracy = M.accuracy(model_outs, labels, average='macro', num_classes=3)
        f1 = M.f1(model_outs, labels, average='macro', num_classes=3)
        self.logger.experiment.log({
            f'{mode}/accuracy':accuracy,
            f'{mode}/f1':f1,
        })
    
    def training_epoch_end(self, step_outputs):
        return self.epoch_end(step_outputs, mode='train')
    
    def validation_epoch_end(self, step_outputs):
        return self.epoch_end(step_outputs, mode='val')
                
    def test_epoch_end(self, step_outputs):
        return self.epoch_end(step_outputs, mode='test')
    
    def configure_optimizers(self, *args, **kwargs):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, batch_size = BATCH_SIZE, shuffle=False, num_workers=4)
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset, batch_size = BATCH_SIZE, shuffle=False, num_workers=4)
            
class ShuffleDatasetIndices(pl.Callback):
    def on_train_epoch_start(self, trainer, pl_module):
        trainer.train_dataloader.dataset.datasets.reset()

    def on_validation_epoch_start(self, trainer, pl_module):
        trainer.val_dataloaders[0].dataset.reset()


    

In [22]:
try:
    logger.experiment.finish()
except:
    pass
logger = pl.loggers.WandbLogger(project='ai4t-DeepLobster')
callbacks = [
    # ShuffleDatasetIndices()
]

In [23]:
BATCH_SIZE = 512

model = LobLightningModule(
    train_dataset = train_dataset,
    val_dataset = val_dataset,
    test_dataset = test_dataset,
    weighted_loss=False,
    lr=1e-2
)

train_dataloader = model.train_dataloader()
train_example_batch = next(iter(train_dataloader))
print({k:v.shape for k, v in train_example_batch.items()})

model_out = (model(**train_example_batch))
print({k:v.shape for k, v in model_out.items()})

{'inputs': torch.Size([512, 40, 100]), 'labels': torch.Size([512])}
{'model_out': torch.Size([512, 3]), 'labels': torch.Size([512])}


In [24]:
from torchinfo import summary
summary(model, (BATCH_SIZE, 40, 100),depth=5)

Layer (type:depth-idx)                        Output Shape              Param #
LobLightningModule                            --                        --
├─LobCNN: 1-1                                 --                        --
│    └─Sequential: 2-1                        [512, 10, 100]            --
│    │    └─Conv1d: 3-1                       [512, 20, 100]            100
│    │    └─Conv1d: 3-2                       [512, 10, 100]            30
│    └─Sequential: 2-2                        [512, 3]                  --
│    │    └─Conv1d: 3-3                       [512, 32, 50]             672
│    │    └─BatchNorm1d: 3-4                  [512, 32, 50]             64
│    │    └─LeakyReLU: 3-5                    [512, 32, 50]             --
│    │    └─Conv1d: 3-6                       [512, 64, 25]             4,160
│    │    └─BatchNorm1d: 3-7                  [512, 64, 25]             128
│    │    └─LeakyReLU: 3-8                    [512, 64, 25]             --
│    │    └─Re

In [25]:
trainer = pl.Trainer(gpus=-1,benchmark=True,logger=logger,
                     max_epochs=10_000,
                    callbacks=callbacks)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type             | Params
-------------------------------------------
0 | model | LobCNN           | 852 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
852 K     Trainable params
0         Non-trainable params
852 K     Total params
3.410     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
logger.experiment.finish()